In [1]:
import pandas as pd
# 데이터 경로
data_path = '/kaggle/input/cat-in-the-dat-ii/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [2]:
# 훈련 데이터와 테스트 데이터 합치기
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1) # 타깃값 제거

In [3]:
def fill(df,feature,g):
    df[feature] = df[feature].fillna(g)
    
    return df

fe1 = ['bin_0' ,'bin_1','bin_2','bin_3','bin_4']
g1 = [0.0, 0.0, 0.0, 'F','Y']

for i,j in zip(fe1,g1):
    fill(all_data,i,j)
#bin_0~4는 대세값인 0으로 처리

In [4]:
fe = ['nom_0' ,'nom_1','nom_2','nom_3','nom_4','ord_0' ,'ord_1','ord_2','ord_3','ord_4','ord_5']
g = ['red','Triangle','Hamster','india','Theremin',1.0,'Novice','Freezing','n', 'N','Fl']

for i,j in zip(fe,g):
    fill(all_data,i,j)

In [5]:
fe = ['day','month']
g = [3.0,8.0]

for i,j in zip(fe,g):
    fill(all_data,i,j)

#ord_0~4는 대세값 처리

In [6]:
fe = ['nom_5' ,'nom_6','nom_7','nom_8','nom_9',]
g = ['other','other','other','other','other']

for i,j in zip(fe,g):
    fill(all_data,i,j)
all_data.info()
#nom_5~9는 각각 따로 처리
all_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 999999
Data columns (total 23 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   bin_0   1000000 non-null  float64
 1   bin_1   1000000 non-null  float64
 2   bin_2   1000000 non-null  float64
 3   bin_3   1000000 non-null  object 
 4   bin_4   1000000 non-null  object 
 5   nom_0   1000000 non-null  object 
 6   nom_1   1000000 non-null  object 
 7   nom_2   1000000 non-null  object 
 8   nom_3   1000000 non-null  object 
 9   nom_4   1000000 non-null  object 
 10  nom_5   1000000 non-null  object 
 11  nom_6   1000000 non-null  object 
 12  nom_7   1000000 non-null  object 
 13  nom_8   1000000 non-null  object 
 14  nom_9   1000000 non-null  object 
 15  ord_0   1000000 non-null  float64
 16  ord_1   1000000 non-null  object 
 17  ord_2   1000000 non-null  object 
 18  ord_3   1000000 non-null  object 
 19  ord_4   1000000 non-null  object 
 20  ord_5   1000000 non-null  obje

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,...,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,india,Theremin,...,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,1.0,0.0,F,N,Red,Triangle,Hamster,Canada,Bassoon,...,745b909d1,other,3.0,Novice,Freezing,n,P,eN,5.0,9.0
3,0.0,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,...,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,Fl,3.0,3.0
4,0.0,0.0,0.0,T,N,Red,Triangle,Hamster,Costa Rica,Theremin,...,other,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0


In [7]:
all_data['bin_3'] = all_data['bin_3'].map({'F':0.0, 'T':1.0})
all_data['bin_4'] = all_data['bin_4'].map({'N':0.0, 'Y':1.0})

ord1dict = {'Novice':0.0, 'Contributor':1.0, 
            'Expert':2.0, 'Master':3.0, 'Grandmaster':4.0}
ord2dict = {'Freezing':0.0, 'Cold':1.0, 'Warm':2.0, 
            'Hot':3.0, 'Boiling Hot':4.0, 'Lava Hot':5.0}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [8]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder() # OrdinalEncoder 객체 생성
# ordinal 인코딩 적용
all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

# 피처별 인코딩 순서 출력
for feature, categories in zip(ord_345, ord_encoder.categories_):
    print(feature)
    print(categories)

ord_3
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o']
ord_4
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
ord_5
['AG' 'AI' 'AU' 'AW' 'Ay' 'BL' 'BX' 'Bx' 'CN' 'CU' 'Cn' 'DI' 'DN' 'DR'
 'DT' 'Dj' 'Dn' 'EC' 'Ey' 'FB' 'FH' 'Fl' 'GZ' 'HF' 'HK' 'HO' 'Hk' 'IA'
 'IS' 'Ib' 'In' 'Io' 'Iq' 'JQ' 'JT' 'Ji' 'Kq' 'LS' 'LY' 'Lo' 'MF' 'MU'
 'MV' 'MX' 'Mg' 'Mq' 'NS' 'NT' 'Nh' 'OM' 'OZ' 'Oe' 'Ox' 'PG' 'PS' 'Pk'
 'Pw' 'QV' 'Qm' 'RB' 'RD' 'RT' 'RV' 'Re' 'Rj' 'Ro' 'Rv' 'Rz' 'SL' 'SS'
 'Sk' 'Sz' 'TH' 'TL' 'TP' 'TZ' 'Tg' 'Ty' 'Tz' 'US' 'UV' 'WC' 'WW' 'Wr'
 'XC' 'XI' 'XM' 'XR' 'XU' 'YJ' 'Yb' 'Yi' 'Yr' 'Zv' 'aA' 'aE' 'al' 'be'
 'cR' 'cY' 'cg' 'cy' 'dh' 'dp' 'eA' 'eN' 'ep' 'fF' 'fO' 'fV' 'fe' 'gK'
 'gL' 'gV' 'gc' 'gj' 'gt' 'hG' 'hT' 'ht' 'hx' 'iS' 'iv' 'ja' 'jf' 'jp'
 'kB' 'kP' 'kT' 'kv' 'lA' 'lR' 'lS' 'ly' 'mD' 'mP' 'mX' 'mi' 'mo' 'nS'
 'ne' 'nf' 'nj' 'nn' 'oI' 'oJ' 'oU' 'oh' 'ok' 'pB' 'pE' 'pT' 'pZ' 'pl'
 'qN' 'qP' 'rA' 'rM' 'rg' 'rl' '

In [9]:
nom_features = ['nom_' + str(i) for i in range(10)] # 명목형 피처

In [10]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder() # OneHotEncoder 객체 생성
# 원-핫 인코딩 적용
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix

<1000000x5434 sparse matrix of type '<class 'numpy.float64'>'
	with 10000000 stored elements in Compressed Sparse Row format>

In [11]:
print(encoded_nom_matrix[0])

  (0, 2)	1.0
  (0, 8)	1.0
  (0, 13)	1.0
  (0, 21)	1.0
  (0, 23)	1.0
  (0, 1086)	1.0
  (0, 2261)	1.0
  (0, 2856)	1.0
  (0, 2993)	1.0
  (0, 3242)	1.0


In [12]:
all_data = all_data.drop(nom_features, axis=1) # 기존 명목형 피처 삭제

In [13]:
date_features  = ['day', 'month'] # 날짜 피처

# 원-핫 인코딩 적용
encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])

all_data = all_data.drop(date_features, axis=1) # 기존 날짜 피처 삭제

encoded_date_matrix

<1000000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 2000000 stored elements in Compressed Sparse Row format>

## 피처 스케일링

In [14]:
from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_' + str(i) for i in range(6)] # 순서형 피처
# min-max 정규화
all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   bin_0   1000000 non-null  float64
 1   bin_1   1000000 non-null  float64
 2   bin_2   1000000 non-null  float64
 3   bin_3   1000000 non-null  float64
 4   bin_4   1000000 non-null  float64
 5   ord_0   1000000 non-null  float64
 6   ord_1   1000000 non-null  float64
 7   ord_2   1000000 non-null  float64
 8   ord_3   1000000 non-null  float64
 9   ord_4   1000000 non-null  float64
 10  ord_5   1000000 non-null  float64
dtypes: float64(11)
memory usage: 91.6 MB


In [15]:
from scipy import sparse

# 인코딩 및 스케일링된 피처 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                              format='csr')

In [16]:
all_data_sprs

<1000000x5464 sparse matrix of type '<class 'numpy.float64'>'
	with 18225434 stored elements in Compressed Sparse Row format>

In [17]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_sprs[:num_train] # 0 ~ num_train - 1행
X_test = all_data_sprs[num_train:] # num_train ~ 마지막 행

y = train['target']

In [18]:
X_train.shape

(600000, 5464)

In [19]:
y.shape

(600000,)

In [20]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

In [21]:
y.shape

(600000,)

## 하이퍼 파라미터 최적화

In [22]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 생성
logistic_model = LogisticRegression(
    C=0.049,
    class_weight={0: 1, 1: 1.42},
    solver='liblinear',
    fit_intercept=True,
    penalty='l2')

logistic_model.fit(X_train,y_train)
# 하이퍼파라미터 값 목록
#lr_params = {'C':[0.049, 0.1, 0.2], 'max_iter':[800, 900, 1000], 
 #            'solver':['liblinear'], 'random_state':[42]}

# 그리드서치 객체 생성
#gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
 #                                        param_grid=lr_params,
  #                                       scoring='roc_auc', # 평가지표
   #                                      cv=5)
# 그리드서치 수행
#gridsearch_logistic_model.fit(X_train, y_train)

#print('최적 하이퍼파라미터:', gridsearch_logistic_model.best_params_)

CPU times: user 12.9 s, sys: 62.3 ms, total: 13 s
Wall time: 13 s


LogisticRegression(C=0.049, class_weight={0: 1, 1: 1.42}, solver='liblinear')

In [23]:
y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

In [24]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7849


In [25]:
# 타깃값 1일 확률 예측
y_preds = logistic_model.predict_proba(X_test)[:,1]

# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('submission.csv')